<a href="https://colab.research.google.com/github/bradbradlin/Machine-Learning-Projects/blob/main/Strategy_by_technical_indicator_v6_4(%2Bve%26_ve)_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To use TI signal and TI

Two classes:  Positive return or Negative return

Weight entropy loss function

Use tf.feature_column



In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import math
import keras.backend as K
from tensorflow import feature_column
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorboard.plugins.hparams import api as hp
print(tf.__version__)

2.4.1


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

%cd /content/gdrive/My Drive/Colab Notebooks/AI trading project

Mounted at /content/gdrive
/content/gdrive/My Drive/Colab Notebooks/AI trading project


In [3]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy')>0.8):
            print("Reached 80% accuracy so cancelling training!")
            self.model.stop_training = True

In [4]:
def tf_dataset(df, target, batch_size, shuffle=True):
    # To convert to tf dataset
    df2 = df.copy()
    #target = df2.pop(TARGET)
    dataset = tf.data.Dataset.from_tensor_slices((dict(df2), target))
    if shuffle:
        dataset = dataset.shuffle(len(df2))
        dataset = dataset.batch(batch_size)
    else:
        dataset = dataset.batch(batch_size)
    return dataset

In [5]:
def get_focal_params(y_pred):
    epsilon = tf.constant(1e-9)
    gamma = tf.constant(3.)
    y_pred = y_pred + epsilon
    pinv = 1./y_pred
    pos_weight_f = (pinv - 1)**gamma
    weight_f = y_pred**gamma
    return pos_weight_f, weight_f

def custom_loss(y_true,y_pred):
    # the custom loss function for imbalance dataset
    y_pred_prob = tf.keras.backend.sigmoid(y_pred)    
    pos_weight_f, weight_f = get_focal_params(y_pred_prob)
    alpha = tf.constant(.35)
    alpha_ = 1 - alpha
    alpha_div = alpha / alpha_
    pos_weight = pos_weight_f * alpha_div
    weight = weight_f * alpha_
    l2 = weight * tf.nn.weighted_cross_entropy_with_logits\
    (labels=y_true, logits=y_pred, pos_weight=weight)
    return l2

In [6]:
# def custom_loss2(y_true,y_pred):
#     class_count_df = raw_df.groupby('return').count()
#     n_0, n_1 = class_count_df.iloc[0, 0], class_count_df.iloc[1,0]
#     w_0 = (n_0 + n_1) / (2.0 * n_0)
#     w_1 = (n_0 + n_1) / (2.0 * n_1)
#     class_weights=torch.tf.tensor([w_0,w_1])

In [7]:
def get_f1(y_true, y_pred): #taken from old keras source code
    # to calculate F1 
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [8]:
def get_normalization_layer(name, dataset):
  # Create a Normalization layer for our feature.
    normalizer = preprocessing.Normalization()

    # Prepare a Dataset that only yields our feature.
    feature_ds = dataset.map(lambda x, y: x[name])

    # Learn the statistics of the data.
    normalizer.adapt(feature_ds)

    return normalizer

In [9]:
def get_category_encoding_layer(name, dataset, dtype, max_tokens=None):
      # Create a StringLookup layer which will turn strings into integer indices
    if dtype == 'string':
        index = preprocessing.StringLookup(max_tokens=max_tokens)
    else:
        index = preprocessing.IntegerLookup(max_values=max_tokens)

    # Prepare a Dataset that only yields our feature
    feature_ds = dataset.map(lambda x, y: x[name])

      # Learn the set of possible values and assign them a fixed integer index.
    index.adapt(feature_ds)

    # Create a Discretization for our integer indices.
    encoder = preprocessing.CategoryEncoding(max_tokens=index.vocab_size())

    # Prepare a Dataset that only yields our feature.
    feature_ds = feature_ds.map(index)

    # Learn the space of possible indices.
    encoder.adapt(feature_ds)

    # Apply one-hot encoding to our indices. The lambda function captures the
    # layer so we can use them, or include them in the functional model later.
    return lambda feature: encoder(index(feature))

In [10]:
callbacks = myCallback()
#@tf.function
def train_model(train_ds, val_ds, epoch, a_inputs, enc_features): 
    L1L2 = tf.keras.regularizers.l1_l2(l1=0.1, l2=0.1)

    tf.keras.backend.clear_session()
    tf.keras.backend.set_floatx('float32')
  
    merged =  tf.keras.layers.concatenate(enc_features)
    dense1 =  tf.keras.layers.Dense(128, activation=tf.nn.relu)(merged)
    dense2 =  tf.keras.layers.Dense(128, activation=tf.nn.relu)(dense1)
    dropout1 = tf.keras.layers.Dropout(0.05)(dense2)
    # dense3 =  tf.keras.layers.Dense(512, activation=tf.nn.relu)(dense2)
    # dense4 =  tf.keras.layers.Dense(1028, activation=tf.nn.relu)(dense3)    
    # dense5 =  tf.keras.layers.Dense(512, activation=tf.nn.relu)(dense4)
    #dense6 =  tf.keras.layers.Dense(256, activation=tf.nn.relu)(dense5)
    #dense7 =  tf.keras.layers.Dense(64, activation=tf.nn.relu)(dense6)
    output =  tf.keras.layers.Dense(num_category, activation=tf.nn.softmax)(dropout1)#)(dense5)
                                   #kernel_regularizer=L1L2,)(dense5)

    model =  tf.keras.models.Model(a_inputs, output)

    opt = tf.keras.optimizers.Adam(
        learning_rate=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,
        name='adam',)
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
    
    model.compile(optimizer=opt,
                  #loss = 'binary_crossentropy', 
                  loss='categorical_crossentropy',
                  #loss= custom_loss,
                  metrics=['accuracy',get_f1])

    history = model.fit(train_ds,
          epochs=epoch, 
          validation_data=val_ds,
          verbose = 2,
          callbacks=[es])
    
    return model, history

In [11]:
def create_fea_col(train_ds):
    all_inputs = []
    encoded_features = []
    # Numeric features.
    for header in CONT_NAMES:
        numeric_col = tf.keras.Input(shape=(1,), name=header)
        normalization_layer = get_normalization_layer(header, train_ds)
        encoded_numeric_col = normalization_layer(numeric_col)
        all_inputs.append(numeric_col)
        encoded_features.append(encoded_numeric_col)

    # Categorical features encoded as int.
    for header in CAT_NAMES:
        categorical_col = tf.keras.Input(shape=(1,), name=header, dtype='string')
        encoding_layer = get_category_encoding_layer(header, train_ds, dtype='string', max_tokens=5)
        encoded_categorical_col = encoding_layer(categorical_col)
        all_inputs.append(categorical_col)
        encoded_features.append(encoded_categorical_col)

    return all_inputs, encoded_features

In [12]:
# load technical indicator from csv
filename = 'tech_GSPC_v2_20201227.csv'
raw_df = pd.read_csv(filename)[200:]
# raw_df2 = pd.read_csv('tech_NDX_v2_20D.csv')[200:-1]
# raw_df3 = pd.read_csv('tech_HSI_v2_20D.csv')[200:-1]
# raw_df4 = pd.read_csv('tech_HSCE_v2_20D.csv')[200:-1]
# raw_df5 = pd.read_csv('tech_DJI_v2_20D.csv')[200:-1]
# raw_df6 = pd.read_csv('tech_GDAXI_v2_20D.csv')[200:-1]
# raw_df7 = pd.read_csv('tech_RUT_v2_20D.csv')[200:-1]
# raw_df8 = pd.read_csv('tech_STI_v2_20D.csv')[200:-1]
# raw_df9 = pd.read_csv('tech_N225_v2_20D.csv')[200:-1]
# raw_df10 = pd.read_csv('tech_FCHI_v2_20D.csv')[200:-1]
# raw_df11 = pd.read_csv('tech_AXJO_v2_20D.csv')[200:-1]

# raw_df = raw_df.append(raw_df2, ignore_index=True)
# raw_df = raw_df.append(raw_df3, ignore_index=True)
# raw_df = raw_df.append(raw_df4, ignore_index=True)
# raw_df = raw_df.append(raw_df5, ignore_index=True)
# raw_df = raw_df.append(raw_df6, ignore_index=True)
# raw_df = raw_df.append(raw_df7, ignore_index=True)
# raw_df = raw_df.append(raw_df8, ignore_index=True)
# raw_df = raw_df.append(raw_df9, ignore_index=True)
# raw_df = raw_df.append(raw_df10, ignore_index=True)
# raw_df = raw_df.append(raw_df11, ignore_index=True)

# VIX_df = pd.read_csv('VIX.csv')
# SKEW_df = pd.read_csv('SKEW.csv')

raw_df.drop(['Unnamed: 0'], axis=1, inplace=True)
raw_df.reset_index(inplace=True)
raw_df.drop(['index'], axis=1, inplace=True)
raw_df['date'] = pd.to_datetime(raw_df['date'])
# VIX_df['date'] = pd.to_datetime(VIX_df['date'])
# SKEW_df['date'] = pd.to_datetime(VIX_df['date'])

In [13]:
#get VIX index
VIX_df = pd.read_csv('^VIX.csv')
VIX_df['Date'] = pd.to_datetime(VIX_df['Date'])
# get probability of recession by new york bank
recess_df = pd.read_excel('prob_recess.xls')
recess_df['Date'] = pd.to_datetime(recess_df['Date'])
recess_df['month'] = recess_df['Date'].dt.month.astype('str')
recess_df['year'] = recess_df['Date'].dt.year.astype('str')
# #get USD dollar index
# stock1 = Ticker('DX-Y.NYB')
# USD_df = stock1.history(period=period).reset_index()
# USD_df['date'] = pd.to_datetime(USD_df['date'])

raw_df = pd.merge(raw_df, VIX_df, how="inner", left_on='date', right_on='Date',suffixes=("","_VIX"))
raw_df['month'] = raw_df['date'].dt.month.astype('str')
raw_df['year'] = raw_df['date'].dt.year.astype('str')
raw_df = pd.merge(raw_df, recess_df, how="inner", left_on=['month','year'], right_on=['month','year'],suffixes=("","_recess"))
# raw_df = pd.merge(raw_df, USD_df, how="inner", left_on='date', right_on='date',suffixes=("","_USDIdx"))

raw_df = raw_df.drop(['Date','Open','High','Low','Adj Close','Volume'
                            ,'month','year','Date_recess','NBER_Rec','10 Year Treasury Yield',
                            '3 Month Treasury Yield','3 Month Treasury Yield (Bond Equivalent Basis)'],axis=1)
#'symbol_USDIdx','open_USDIdx','high_USDIdx','volume_USDIdx','low_USDIdx','adjclose_USDIdx'],axis=1)

raw_df = raw_df.rename(columns={'Close': 'Close_VIX'})

In [14]:
#choose forcast days
day = 10

day_dict = {1: 'marking date return 1D', 5: 'marking date return 5D', 10: 'marking date return 10D'}

week_dict = {1: 'weekday1', 5: 'weekday5', 10: 'weekday10'}
month_dict = {1: 'month1', 5: 'month5', 10: 'month10'}

# to_catergorical input factors
CAT_NAMES = list(raw_df.loc[:, raw_df.dtypes == 'int64'].columns)[:-3]
CAT_NAMES.extend(['volatility_bbhi','volatility_bbli','volatility_kchi', 
                  'volatility_kcli', 'trend_psar_up_indicator', 'trend_psar_down_indicator',
                 week_dict[day], month_dict[day]])
# CAT_NAMES = ['CDLHAMMER',
#  'CDLHIKKAKE',
#  'trend_psar_up_indicator',
#  'trend_psar_down_indicator',
#  'CDLSEPARATINGLINES',
#  'CDLHARAMI',
#  'CDL3OUTSIDE',
#  'CDLINVERTEDHAMMER',
#  'CDL3INSIDE',
#  'CDLADVANCEBLOCK',
#  'CDLEVENINGSTAR',
#  'CDLXSIDEGAP3METHODS',
#  'CDLSHOOTINGSTAR',
#  'CDLTHRUSTING',
#  'CDLTASUKIGAP']

print(CAT_NAMES)

a = list(raw_df.loc[:, raw_df.dtypes == 'float64'].columns)
CONT_NAMES = [x for x in a if x not in ['open','volume','low','high','close','adjclose', 
                                        'volatility_bbhi', 'volatility_bbli', 'volatility_kchi', 
                                        'volatility_kcli', 'trend_psar_up_indicator', 'trend_psar_down_indicator',
                                       'weekday1','month1','weekday5','month5','weekday10','month10','marking date return 1D','marking date return 5D','marking date return 10D']]

# raw_df[CAT_NAMES]= raw_df[CAT_NAMES].replace(-1, 'Sell')
# raw_df[CAT_NAMES]= raw_df[CAT_NAMES].replace(0, 'NA')
# raw_df[CAT_NAMES]= raw_df[CAT_NAMES].replace(1, 'Buy')
raw_df[CAT_NAMES] = raw_df[CAT_NAMES].astype('string')
#raw_df = raw_df.dropna(how="any", axis=0)


#convert to catergorical output
TARGET = ['negative', 'no' , 'positive']
#y = raw_df.apply(lambda x: 1 if x[day_dict[day]]>=0.005 else 0, axis=1)
y = raw_df.apply(lambda x: 2 if x[day_dict[day]]>=0.003 else (0 if x[day_dict[day]]<=-0.003 else 1), axis=1)
raw_df[TARGET] = pd.DataFrame(to_categorical(y), columns=TARGET)
raw_df[TARGET] = raw_df[TARGET].astype('int')
print(raw_df[TARGET].value_counts())
# raw_df = raw_df.fillna(0)
#raw_df = raw_df.dropna(how="any", axis=0)
# raw_df = pd.merge(raw_df, VIX_df, how='left', left_on='date', right_on='date')
# raw_df = pd.merge(raw_df, SKEW_df, how='left', left_on='date', right_on='date')
raw_df2 = raw_df.copy()

Input_col = []
Input_col.extend(CAT_NAMES)
Input_col.extend(CONT_NAMES)
Input_col.extend(TARGET)
Input_col.append('date')
Input_col.append('symbol')

raw_df = raw_df[raw_df.columns[raw_df.columns.isin(Input_col)]]
raw_df = raw_df.fillna('0')
org_df = raw_df.copy()

['CDL2CROWS', 'CDL3BLACKCROWS', 'CDL3INSIDE', 'CDL3LINESTRIKE', 'CDL3OUTSIDE', 'CDL3STARSINSOUTH', 'CDL3WHITESOLDIERS', 'CDLABANDONEDBABY', 'CDLADVANCEBLOCK', 'CDLBELTHOLD', 'CDLBREAKAWAY', 'CDLCLOSINGMARUBOZU', 'CDLCONCEALBABYSWALL', 'CDLDARKCLOUDCOVER', 'CDLDOJI', 'CDLDOJISTAR', 'CDLDRAGONFLYDOJI', 'CDLENGULFING', 'CDLEVENINGDOJISTAR', 'CDLEVENINGSTAR', 'CDLGAPSIDESIDEWHITE', 'CDLGRAVESTONEDOJI', 'CDLHAMMER', 'CDLHANGINGMAN', 'CDLHARAMI', 'CDLHARAMICROSS', 'CDLHIGHWAVE', 'CDLHIKKAKE', 'CDLHIKKAKEMOD', 'CDLHOMINGPIGEON', 'CDLIDENTICAL3CROWS', 'CDLINNECK', 'CDLINVERTEDHAMMER', 'CDLKICKING', 'CDLLADDERBOTTOM', 'CDLLONGLEGGEDDOJI', 'CDLMARUBOZU', 'CDLMATCHINGLOW', 'CDLMATHOLD', 'CDLMORNINGDOJISTAR', 'CDLMORNINGSTAR', 'CDLONNECK', 'CDLPIERCING', 'CDLRICKSHAWMAN', 'CDLRISEFALL3METHODS', 'CDLSEPARATINGLINES', 'CDLSHOOTINGSTAR', 'CDLSPINNINGTOP', 'CDLSTICKSANDWICH', 'CDLTAKURI', 'CDLTASUKIGAP', 'CDLTHRUSTING', 'CDLTRISTAR', 'CDLUNIQUE3RIVER', 'CDLUPSIDEGAP2CROWS', 'CDLXSIDEGAP3METHODS', 'vol

In [15]:
batch_size = 32
eval_start = '20070101'
val_start = '20180101'
tf.keras.backend.clear_session()
tf.random.set_seed(51)
np.random.seed(51)

raw_df = raw_df[raw_df['date']>=eval_start].reset_index()
train_df = raw_df[raw_df['date']<val_start]
train_df = train_df.set_index('date')
train_df = train_df.drop(['index'], axis=1)
train_df[CONT_NAMES] = train_df[CONT_NAMES].astype('float32')
train_df['symbol'] = train_df['symbol'].astype('string')
train_df[CAT_NAMES] = train_df[CAT_NAMES].astype('string')

val_df = raw_df[raw_df['date']>=val_start]
val_df = val_df.set_index('date')
val_df = val_df.drop(['index'], axis=1)
val_df[CONT_NAMES] = val_df[CONT_NAMES].astype('float32')
val_df['symbol'] = val_df['symbol'].astype('string')
val_df[CAT_NAMES] = val_df[CAT_NAMES].astype('string')

temp_train = train_df.loc[:, Input_col[:len(CAT_NAMES)+len(CONT_NAMES)]]
temp_val = val_df.loc[:, Input_col[:len(CAT_NAMES)+len(CONT_NAMES)]]
# only top15 important fesatures
COL = []
COL.extend(CAT_NAMES)
COL.extend(CONT_NAMES)

# CAT_NAMES = ['month10']
# CONT_NAMES = ['volatility_kcw','volume_nvi', 'trend_visual_ichimoku_b','volatility_atr',
# 'volume_obv', 'trend_adx', 'trend_kst_sig', 'trend_mass_index',
# 'volume_mfi', 'trend_aroon_ind', 'volatility_dcw', 'trend_adx_neg', 'trend_kst_diff']
   
temp_train = temp_train.loc[:,COL]
temp_val = temp_val.loc[:,COL]


train1_ds = tf_dataset(temp_train, train_df[TARGET], batch_size, True)
train_ds = train1_ds.take(int(len(train_df)/batch_size*0.9))
# # train_dev dataset for validate variance of model
train_dev_ds = train1_ds.skip(int(len(train_df)/batch_size*0.9))
val_ds = tf_dataset(temp_val, val_df[TARGET], batch_size, False)
train2_ds = tf_dataset(temp_train, train_df[TARGET], batch_size, False)

allinputs, encodedfeatures = create_fea_col(train_ds)

print(train_df[TARGET].value_counts())

negative  no  positive
0         0   1           1539
1         0   0            956
0         1   0            274
dtype: int64


In [16]:
#@title
# num_category = len(TARGET)
# model2, history = train_model(train_ds, val_ds, 500, allinputs, encodedfeatures)
# model2.evaluate(train_dev_ds)
# model2.evaluate(val_ds)

In [17]:
#@title
#model2.summary()

In [18]:
#@title
# y_true = []
# y_pred = []
# val_inputs = []
# thres = 0.5
# #index = '^GSPC'

# for x, y in val_ds:
#     for j in y:
#         y_true.append(j.numpy())

# y_pred = pd.DataFrame(model2.predict(val_ds))
# #y_pred = pd.DataFrame(np.argmax(model2.predict(val_ds), axis=1))
# y_true = pd.DataFrame(np.argmax(y_true, axis=1))
# result = pd.concat([y_true, y_pred], axis=1, ignore_index=True)
# result.columns=['true_return', 'pred_neg', 'na','pred_pos']
# result['true(+/-)'] = result.apply(lambda x: '+ve' if x['true_return']==2 else ('-ve' if x['true_return']==0 else'no') , axis=1)
# result['pred(+/-)']  = result.apply(lambda x: '+ve' if x['pred_pos']>thres else ('-ve' if x['pred_neg']>thres else'no'), axis=1)

# # result['true(+/-)'] = y_true.apply(lambda x: '++ve' if x[0]==3 else ('+ve' if x[0]==2 else ('no' if x[0]==1 else ('-ve' if x[0]==0 else '--ve'))), axis=1)
# # result['pred(+/-)']  = y_pred.apply(lambda x: '++ve' if x[0]==3 else ('+ve' if x[0]==2 else ('no' if x[0]==1 else ('-ve' if x[0]==0 else '--ve'))), axis=1)
# result.to_excel('result.xls')
# #result['acc'] = result.apply(lambda x: 1 if result['true(+/-)'] == result['pred(+/-)']  else 0, axis=1)
# print(result['pred(+/-)'].value_counts())

# from sklearn.metrics import classification_report
# print(CAT_NAMES)
# print(classification_report(result['true(+/-)'] , result['pred(+/-)']))

# result.to_csv('output_pos_val.csv')
# result.to_excel('output_pos_val.xls')

In [19]:
#@title
# y_true = []
# y_pred = []
# val_inputs = []
# thres = 0.5
# #index = '^GSPC'

# for x, y in train2_ds:
#     for j in y:
#         y_true.append(j.numpy())

# y_pred = pd.DataFrame(model2.predict(train2_ds))
# #y_pred = pd.DataFrame(np.argmax(model2.predict(val_ds), axis=1))
# y_true = pd.DataFrame(np.argmax(y_true, axis=1))
# result = pd.concat([y_true, y_pred], axis=1, ignore_index=True)
# result.columns=['true_return', 'pred_neg', 'na','pred_pos']
# result['true(+/-)'] = result.apply(lambda x: '+ve' if x['true_return']==2 else ('-ve' if x['true_return']==0 else'no') , axis=1)
# result['pred(+/-)']  = result.apply(lambda x: '+ve' if x['pred_pos']>thres else ('-ve' if x['pred_neg']>thres else'no'), axis=1)

# # result['true(+/-)'] = y_true.apply(lambda x: '++ve' if x[0]==3 else ('+ve' if x[0]==2 else ('no' if x[0]==1 else ('-ve' if x[0]==0 else '--ve'))), axis=1)
# # result['pred(+/-)']  = y_pred.apply(lambda x: '++ve' if x[0]==3 else ('+ve' if x[0]==2 else ('no' if x[0]==1 else ('-ve' if x[0]==0 else '--ve'))), axis=1)
# #result.to_excel('result.xls')
# #result['acc'] = result.apply(lambda x: 1 if result['true(+/-)'] == result['pred(+/-)']  else 0, axis=1)
# print(result['pred(+/-)'].value_counts())

# from sklearn.metrics import classification_report
# print(CAT_NAMES)
# print(classification_report(result['true(+/-)'] , result['pred(+/-)']))

# result.to_csv('output_pos.csv')

In [20]:
#@title
# import matplotlib.image  as mpimg
# import matplotlib.pyplot as plt

# #-----------------------------------------------------------
# # Retrieve a list of list results on training and test data
# # sets for each training epoch
# #-----------------------------------------------------------
# mae=history.history['get_f1']
# loss=history.history['loss']
# v_loss=history.history['val_loss']
# v_mae = history.history['val_get_f1']
# epochs=range(len(loss)) # Get number of epochs

# #------------------------------------------------
# # Plot MAE and Loss
# #------------------------------------------------
# plt.plot(epochs, mae, 'r')
# plt.plot(epochs, loss, 'b')
# plt.title('Train F1 and Loss')
# plt.xlabel("Epochs")
# plt.ylabel("F1")
# plt.legend(["F1", "Loss"])

# plt.figure()

# epochs_zoom = epochs[:]
# mae_zoom = v_mae[:]
# loss_zoom = v_loss[:]

# #------------------------------------------------
# # Plot Zoomed MAE and Loss
# #------------------------------------------------
# plt.plot(epochs_zoom, mae_zoom, 'r')
# plt.plot(epochs_zoom, loss_zoom, 'b')
# plt.title('Val F1 and Loss')
# plt.xlabel("Epochs")
# plt.ylabel("F1")
# plt.legend(["F1", "Loss"])

# plt.figure()

Testing expanding windows

In [21]:
model1 = tf.keras.models.load_model('saved_model/my_model')

# Check its architecture
model1.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
CDL2CROWS (InputLayer)          [(None, 1)]          0                                            
__________________________________________________________________________________________________
CDL3BLACKCROWS (InputLayer)     [(None, 1)]          0                                            
__________________________________________________________________________________________________
CDL3INSIDE (InputLayer)         [(None, 1)]          0                                            
__________________________________________________________________________________________________
CDL3LINESTRIKE (InputLayer)     [(None, 1)]          0                                            
______________________________________________________________________________________________

In [22]:
from pandas.tseries.offsets import *
BDay()
# #create rolling windows dataset
n_train = 5*(52) # training period 
val_size = 10 # validation period (next n days)
window_shift = 10 # period for re-train model
n_records = len(raw_df[raw_df['symbol']=='^GSPC'])
fq = 'B'
batch_size = 32
num_category = len(TARGET)
ts_date= []
te_date = []
vt_date = []
ve_date = []
td_acc = []
t_acc = []
v_acc = []
t_F1 = []
td_F1 = []
v_F1 = []
y_true = []
y_pred = pd.DataFrame({})


train_start =  pd.Timestamp('2016-01-02')
for j, i in enumerate(np.arange(1, 255, window_shift)):
    #split train and test
    
    train_range = pd.date_range(train_start, periods=n_train, freq=fq)
    test_start = train_range[-1]+BDay()
    test_range = pd.date_range(test_start, periods=val_size, freq=fq)
    
    train_df, val_df = raw_df[raw_df['date'].isin(train_range)], raw_df[raw_df['date'].isin(test_range)]
    train_df[CONT_NAMES] = train_df[CONT_NAMES].astype('float32')
    val_df[CONT_NAMES] = val_df[CONT_NAMES].astype('float32')
    
    #y_train, y_val = raw_df[raw_df['date'].isin(train_range)][TARGET], raw_df[raw_df['date'].isin(test_range)][TARGET]
    print(i, 'Training window: {} - {}'.format(train_range[0], train_range[-1]))
    if len(val_df)==0:
        # train_start = train_range[0]+BDay()
        n_train = n_train + window_shift
        continue
    #print('train=%d, test=%d' % (len(train_df), len(test_df)))
    temp_train = train_df.iloc[:, :len(CAT_NAMES)+len(CONT_NAMES)].reset_index()
    temp_train = train_df.loc[:, COL].reset_index()
    temp_train = temp_train.drop(['index'], axis=1)
    temp_val = val_df.iloc[:, :len(CAT_NAMES)+len(CONT_NAMES)].reset_index()
    temp_val = val_df.loc[:, COL].reset_index()
    temp_val = temp_val.drop(['index'], axis=1)
    
    train1_ds = tf_dataset(temp_train, train_df[TARGET], batch_size, True)
    train_ds = train1_ds.take(int(len(train_df)/batch_size*0.9))
    # # train_dev dataset for validate variance of model
    train_dev_ds = train1_ds.skip(int(len(train_df)/batch_size*0.9))
    val_ds = tf_dataset(temp_val, val_df[TARGET], batch_size, False)

#     train1_ds = tf_dataset(train_df.loc[:, Input_col[:-1]], train_df[TARGET], batch_size, True)
#     train_ds = train1_ds.take(int(len(train_df)/batch_size*0.9))
#     # # train_dev dataset for validate variance of model
#     train_dev_ds = train1_ds.skip(int(len(train_df)/batch_size*0.9))
#     val_ds = tf_dataset(val_df.loc[:, Input_col[:-1]], val_df[TARGET], batch_size, False)
    
    #allinputs, encodedfeatures = create_fea_col(train_ds)

    #model training
    #model1, history = train_model(train_ds, train_dev_ds, 200, allinputs, encodedfeatures)
    #model1 =  train_model(train_ds, val_ds, 50, encoded_features)
    
    loss, acc, F1 = model1.evaluate(train_ds, verbose=0)
    print(i, 'Model train accuracy: {:5.2f}% , F1: {:5.2f}'.format(100 * acc, F1))

    
    #model validation
    loss, acc, F1= model1.evaluate(train_dev_ds, verbose=0)
    print(i, 'Model train-dev accuracy: {:5.2f}% , F1: {:5.2f}'.format(100 * acc, F1))

    
    print('Val window: {} - {}'.format(test_range[0], test_range[-1]))

    loss, acc, F1 = model1.evaluate(val_ds, verbose=0)
    y_pred = y_pred.append(pd.DataFrame(model1.predict(val_ds)), ignore_index=True, sort=False)
    #print(y_pred)
    
    for x, y in val_ds:
        for k in y:
            y_true.append(k.numpy())
            t_acc.append(acc)
            t_F1.append(F1)
            td_acc.append(acc)
            td_F1.append(F1)
            v_acc.append(acc)
            v_F1.append(F1)
            ts_date.append(train_start)
            te_date.append(train_range[-1])
            vt_date.append(test_start)
            ve_date.append(test_range[-1])
            test_start = test_start + BDay()

    print(i, 'Model val accuracy: {:5.2f}% , F1: {:5.2f} \n'.format(100 * acc, F1))
    
    
    # train_start = train_range[0]+BDay(window_shift)
    n_train = n_train + window_shift

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3069: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


1 Training window: 2016-01-04 00:00:00 - 2016-12-30 00:00:00
1 Model train accuracy: 89.80% , F1:  0.79
1 Model train-dev accuracy: 90.00% , F1:  0.64
Val window: 2017-01-02 00:00:00 - 2017-01-13 00:00:00
1 Model val accuracy: 62.50% , F1:  0.56 

11 Training window: 2016-01-04 00:00:00 - 2017-01-13 00:00:00


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3069: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


11 Model train accuracy: 87.76% , F1:  0.77
11 Model train-dev accuracy: 94.12% , F1:  0.86
Val window: 2017-01-16 00:00:00 - 2017-01-27 00:00:00
11 Model val accuracy: 66.67% , F1:  0.22 

21 Training window: 2016-01-04 00:00:00 - 2017-01-27 00:00:00


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3069: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


...

231 Model train accuracy: 84.13% , F1:  0.76
231 Model train-dev accuracy: 81.13% , F1:  0.73
Val window: 2017-11-20 00:00:00 - 2017-12-01 00:00:00
231 Model val accuracy: 88.89% , F1:  0.89 

241 Training window: 2016-01-04 00:00:00 - 2017-12-01 00:00:00


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3069: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


241 Model train accuracy: 84.35% , F1:  0.76
241 Model train-dev accuracy: 81.36% , F1:  0.71
Val window: 2017-12-04 00:00:00 - 2017-12-15 00:00:00
241 Model val accuracy: 100.00% , F1:  1.00 

251 Training window: 2016-01-04 00:00:00 - 2017-12-15 00:00:00


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3069: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


251 Model train accuracy: 85.60% , F1:  0.77
251 Model train-dev accuracy: 83.58% , F1:  0.72
Val window: 2017-12-18 00:00:00 - 2017-12-29 00:00:00
251 Model val accuracy: 100.00% , F1:  1.00 



In [23]:
thres = 0.5

a = pd.DataFrame(ts_date, columns=['train start date'])
b = pd.DataFrame(te_date, columns=['train end date'])
c = pd.DataFrame(t_acc, columns=['train accuracy'])
d = pd.DataFrame(vt_date, columns=['val start date'])
e = pd.DataFrame(ve_date, columns=['val end date'])
f = pd.DataFrame(v_acc, columns=['val accuracy'])
g = pd.DataFrame(td_acc, columns=['train-dev accuracy'])
h = pd.DataFrame(y_true, columns=['true_' + sub for sub in TARGET])
i = y_pred
i.columns = ['pred_-ve', 'pred_no', 'pred_+ve']

out_df = pd.concat([a, b, c, g, d, e, f, h, i], axis=1)
out_df['true(+/-)'] = out_df.apply(lambda x: '+ve' if x['true_positive']==1 else ('-ve' if x['true_negative']==1 else'no') , axis=1)
out_df['pred(+/-)']  = out_df.apply(lambda x: '+ve' if x['pred_+ve']>thres else ('-ve' if x['pred_-ve']>thres else'no'), axis=1)

out_df.tail(20)

out_df.to_excel('output.xls')

In [24]:
out_df.head(2)

,train start date,train end date,train accuracy,train-dev accuracy,val start date,val end date,val accuracy,true_negative,true_no,true_positive,pred_-ve,pred_no,pred_+ve,true(+/-),pred(+/-)
0,2016-01-02,2016-12-30,0.625,0.625,2017-01-02,2017-01-13,0.625,0,0,1,0.225112,0.035607,0.739281,+ve,+ve
1,2016-01-02,2016-12-30,0.625,0.625,2017-01-03,2017-01-13,0.625,1,0,0,0.312698,0.103336,0.583965,-ve,+ve


In [25]:
from sklearn.metrics import classification_report
print(CAT_NAMES)
print(classification_report(out_df['true(+/-)'] , out_df['pred(+/-)']))
print(out_df['pred(+/-)'].value_counts())
print(out_df['true(+/-)'].value_counts())

['CDL2CROWS', 'CDL3BLACKCROWS', 'CDL3INSIDE', 'CDL3LINESTRIKE', 'CDL3OUTSIDE', 'CDL3STARSINSOUTH', 'CDL3WHITESOLDIERS', 'CDLABANDONEDBABY', 'CDLADVANCEBLOCK', 'CDLBELTHOLD', 'CDLBREAKAWAY', 'CDLCLOSINGMARUBOZU', 'CDLCONCEALBABYSWALL', 'CDLDARKCLOUDCOVER', 'CDLDOJI', 'CDLDOJISTAR', 'CDLDRAGONFLYDOJI', 'CDLENGULFING', 'CDLEVENINGDOJISTAR', 'CDLEVENINGSTAR', 'CDLGAPSIDESIDEWHITE', 'CDLGRAVESTONEDOJI', 'CDLHAMMER', 'CDLHANGINGMAN', 'CDLHARAMI', 'CDLHARAMICROSS', 'CDLHIGHWAVE', 'CDLHIKKAKE', 'CDLHIKKAKEMOD', 'CDLHOMINGPIGEON', 'CDLIDENTICAL3CROWS', 'CDLINNECK', 'CDLINVERTEDHAMMER', 'CDLKICKING', 'CDLLADDERBOTTOM', 'CDLLONGLEGGEDDOJI', 'CDLMARUBOZU', 'CDLMATCHINGLOW', 'CDLMATHOLD', 'CDLMORNINGDOJISTAR', 'CDLMORNINGSTAR', 'CDLONNECK', 'CDLPIERCING', 'CDLRICKSHAWMAN', 'CDLRISEFALL3METHODS', 'CDLSEPARATINGLINES', 'CDLSHOOTINGSTAR', 'CDLSPINNINGTOP', 'CDLSTICKSANDWICH', 'CDLTAKURI', 'CDLTASUKIGAP', 'CDLTHRUSTING', 'CDLTRISTAR', 'CDLUNIQUE3RIVER', 'CDLUPSIDEGAP2CROWS', 'CDLXSIDEGAP3METHODS', 'vol

In [26]:
#@title
# num_category = len(TARGET)
# train_range = pd.date_range('2019-12-23', periods=254, freq='B')
# print(train_range)
# train_df = raw_df[raw_df['date'].isin(train_range)]
# train_df[CONT_NAMES] = train_df[CONT_NAMES].astype('float32')
# temp_train = train_df.loc[:, COL].reset_index()
# #temp_train = temp_train.drop(['index','date','level_0','symbol','CDLXSIDEGAP3METHODS'], axis=1)
# train1_ds = tf_dataset(temp_train, train_df[TARGET], batch_size, True)
# train_ds = train1_ds.take(int(len(train_df)/batch_size*0.9))
# # # train_dev dataset for validate variance of model
# train_dev_ds = train1_ds.skip(int(len(train_df)/batch_size*0.9))
# allinputs, encodedfeatures = create_fea_col(train_ds)
# model1, history = train_model(train_ds, train_dev_ds, 300, allinputs, encodedfeatures)

In [27]:
#@title
# val_df = raw_df[raw_df['date']=='2020-12-24']
# val_df[CONT_NAMES] = val_df[CONT_NAMES].astype('float32')
# temp_val = val_df.loc[:, COL].reset_index()
# #temp_val = temp_val.drop(['index','date','level_0','symbol'], axis=1)
# temp_val = temp_val.drop('index', axis=1)
# val_ds = tf_dataset(temp_val, val_df[TARGET], batch_size, False)
# model1.predict(val_ds)

# 